<a href="https://colab.research.google.com/github/BHARADHWAJ27/My-Zoom-Transformer-Based-Model-for-Feedback-Validation/blob/main/My_Zoom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets gradio scikit-learn pandas

In [ ]:
import pickle
import torch
from transformers import BertTokenizer
import pandas as pd
import gradio as gr
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# REMOVED: File merging - The files were not found. Using the already loaded dataframe.

In [ ]:
df=pd.read_csv("/content/data_set.csv")
df.head(50)

,l,reason,label
0,streaming functionality is awful.,worst app for streaming,1
1,also raised hands are not brought to the front...,raised hands are not brought to the front of t...,1
2,your data is not safe on this app.,data is not safe,1
3,why is the app not running on my samsung tv.,unable to open on android tv,1
4,browser access does not work with chrome on an...,browser access does not work with chrome on an...,1
5,app fails to work offline despite being up to ...,unable to access the app without internet conn...,1
6,i am not able to get video,want to access video,1
7,best streaming channel ever,best streaming service,1
8,android app does not allow direct screen shari...,app freezes frequently,0
9,it take high speed network.very bad app,app requires high speed internet,1


In [ ]:
# 3. PREPROCESSING
from transformers import BertTokenizer
from datasets import Dataset

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(example):
    return tokenizer(example["l"], example["reason"], truncation=True)


dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize, batched=True)

# Train-test split
dataset = dataset.train_test_split(test_size=0.2)

Map:   0%|          | 0/4122 [00:00<?, ? examples/s]

In [ ]:
# 4. MODEL DEFINITION
from transformers import BertForSequenceClassification, DataCollatorWithPadding
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 5. TRAINING ARGUMENTS
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # Disable logging to Weights & Biases
)

# 6. TRAINER SETUP
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-3360301152.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 7. TRAIN THE MODEL
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.113400,0.172090
2,0.100500,0.125584
3,0.123100,0.154181


TrainOutput(global_step=1239, training_loss=0.10689535283778973, metrics={'train_runtime': 178.5101, 'train_samples_per_second': 55.409, 'train_steps_per_second': 6.941, 'total_flos': 178563503830500.0, 'train_loss': 0.10689535283778973, 'epoch': 3.0})

In [ ]:
# 8. EVALUATE
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
predictions = trainer.predict(dataset["test"])
preds = np.argmax(predictions.predictions, axis=-1)

print(classification_report(dataset["test"]["label"], preds))
print(confusion_matrix(dataset["test"]["label"], preds))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97       421
           1       0.96      0.97      0.97       404

    accuracy                           0.97       825
   macro avg       0.97      0.97      0.97       825
weighted avg       0.97      0.97      0.97       825

[[405  16]
 [ 12 392]]


It looks like the `trainer` object was not defined before this cell was executed. Please run the cells in sections 4, 5, and 6 before running this cell to ensure that the `trainer` object is properly initialized.

In [ ]:
import pickle
import torch

# Save the trained model using pickle to a known path
model_path = "/content/trained._model.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(model, f)

print(f"Model pickled successfully at: {model_path}")

Model pickled successfully at: /content/trained._model.pkl


In [ ]:
# 9. DEPLOY WITH GRADIO
def classify_feedback(feedback, reason):
    inputs = tokenizer(feedback, reason, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    pred = torch.argmax(logits, axis=1).item()
    return "Aligned ✅" if pred == 1 else "Not Aligned ❌"

# Gradio UI
interface = gr.Interface(
    fn=classify_feedback,
    inputs=[gr.Textbox(label="User Feedback"), gr.Dropdown(choices=df['reason'].unique().tolist(), label="Dropdown Reason")],
    outputs="l",
    title="My Zoom Feedback Validator",
    description="Check if the user's feedback matches the dropdown reason"
)

interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4fa31df293c022c117.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Load the pickled model from the correct path
model_path = "/content/trained._model.pkl"
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

# Move the model to the appropriate device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)
loaded_model.eval() # Set the model to evaluation mode

# Initialize the tokenizer (needs to be the same as the one used for training)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load the original data to get the reasons for the dropdown
# Assuming the original data is available at "/content/train.csv" based on previous cells
try:
    df_reasons = pd.read_csv("/content/data_set.csv")
    reasons_list = df_reasons['reason'].unique().tolist()
except FileNotFoundError:
    print("Could not find the original data file '/content/data_set.csv'. Please ensure it's in the correct path.")
    reasons_list = ["Reason not loaded (file not found)"]

# 9. DEPLOY WITH GRADIO
def classify_feedback(feedback, reason):
    inputs = tokenizer(feedback, reason, return_tensors="pt", truncation=True, padding=True)
    # Move input tensors to the specified device
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = loaded_model(**inputs)
    logits = outputs.logits
    pred = torch.argmax(logits, axis=1).item()
    return "Aligned ✅" if pred == 1 else "Not Aligned ❌"

# Gradio UI
interface = gr.Interface(
    fn=classify_feedback,
    inputs=[gr.Textbox(label="User Feedback"), gr.Dropdown(choices=reasons_list, label="Dropdown Reason")],
    outputs="text",
    title="My Zoom Feedback Validator",
    description="Check if the user's feedback matches the dropdown reason"
)

interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e3a99b1f296c4a88b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
